# **Load Dataset**

In [ ]:
!gdown 1kTwEZ937PJ8tQOL8BmxYAtQwCXyTYtzp

Downloading...
From: https://drive.google.com/uc?id=1kTwEZ937PJ8tQOL8BmxYAtQwCXyTYtzp
To: /content/uniform_data.zip
100% 5.85M/5.85M [00:00<00:00, 68.3MB/s]


In [ ]:
!unzip 'uniform_data.zip'

Archive:  uniform_data.zip
  inflating: uniform_data_100.npy    
  inflating: uniform_data_200.npy    
  inflating: uniform_data_25.npy     
  inflating: uniform_data_400.npy    
  inflating: uniform_data_50.npy     


# **Imports**

In [ ]:
import pandas as pd
import pymc as pm
import numpy as np
import scipy.stats as stats
import copy

from google.colab import files
from tqdm.notebook import tqdm, trange
from pytensor.tensor import TensorVariable, log, exp

# **Simulations**

In [ ]:
# Lista dos tamanhos de amostra que foram utilizados anteriormente.
sizes = [25, 50, 100, 200, 400]

# Dicionário para armazenar os dados carregados dos arquivos numpy.
uniform_data = {}
for size in sizes:
  with open(f'uniform_data_{size}.npy', 'rb') as f:
    # Carrega os dados do arquivo numpy e os armazena no dicionário.
    uniform_data[size] = np.load(f)

In [ ]:
#@title Create Experiments
def logp(x: TensorVariable, beta: TensorVariable) -> TensorVariable:
    return beta/x - (1/beta) * (exp(beta/x) - 1) - 2*log(x)


def ppf(q: float, beta: float) -> float:
    return np.divide( beta, np.log(1 - beta * np.log(q)) )


def create_betas(target_beta, unifom_data,
                 size, seed = 42, alpha = .05,
                 beta_guess = 1e-2, data_guess = 1e-2,
                 a_gamma = 1, b_gamma = 1e-3,
                 a_unif = 0, b_unif = 1e+3,
                 burn  = 5000, trace_size = 5000,
                 step_size = 5):
    # Cria uma cópia profunda dos dados uniformes gerados.
    new_data = copy.deepcopy(uniform_data)

    # Transforma os dados uniformes gerados usando a função PPF.
    for key in tqdm(new_data, desc='Transform Dataset'):
      for idx, data in enumerate(new_data[key]):
        sample = [ppf(x, target_beta) for x in data]
        new_data[key][idx] = sample


    gamma_info = []
    unif_info = []

    # Estimação dos parâmetros beta para a distribuição Gamma e Uniforme.
    for sample in tqdm(new_data[size], desc='Estimating Betas'):

      # Estimação para a distribuição Gamma.
      with pm.Model():
          beta = pm.Gamma('beta', a_gamma, b_gamma)
          data = pm.CustomDist('data', beta, logp=logp, observed=sample)

          start = {'beta': beta_guess, 'data': data_guess}
          step = pm.Metropolis(step_scale=step_size)
          gam_trace = pm.sample(trace_size, tune=burn, cores=4,
                                random_seed=seed, initvals=start,
                                step=step, chains=1, progressbar=False)

      gam_beta = np.mean(gam_trace.posterior.beta[0].to_numpy())
      gam_values = pm.hdi(gam_trace, hdi_prob=(1 - alpha))
      gam_values = gam_values.beta.to_numpy()
      gam_values = np.append(gam_values, gam_beta)

      gamma_info.append(gam_values)

      # Estimação para a distribuição Uniforme.
      with pm.Model():
            beta = pm.Uniform('beta', a_unif, b_unif)
            data = pm.CustomDist('data', beta, logp=logp, observed=sample)

            start = {'beta': beta_guess, 'data':data_guess}
            step = pm.Metropolis(step_scale=step_size)
            unif_trace = pm.sample(trace_size, tune=burn, cores=4,
                                  random_seed=seed, initvals=start,
                                  step=step, chains=1, progressbar=False)

      unif_beta = np.mean(unif_trace.posterior.beta[0].to_numpy())
      unif_values = pm.hdi(unif_trace, hdi_prob=(1 - alpha))
      unif_values = unif_values.beta.to_numpy()
      unif_values = np.append(unif_values, unif_beta)

      unif_info.append(unif_values)

    return gamma_info, unif_info

In [ ]:
size = 100 #@param
beta = .125 #@param
beta = float(beta)

# Executa a função create_betas para estimar os parâmetros beta para as distribuições Gamma e Uniforme.
gamma_betas, unif_betas =  create_betas(beta, uniform_data, size)

# Redimensiona as listas de resultados para serem unidimensionais.
gamma_betas = np.squeeze(gamma_betas)
unif_betas = np.squeeze(unif_betas)

# Combina os resultados em um array bidimensional.
data = np.append(gamma_betas, unif_betas, axis=1)

# Cria um DataFrame pandas com os resultados.
df = pd.DataFrame(data, columns=['Lower-Gamma', 'Upper-Gamma', 'Gamma',
                                 'Lower-Uniform', 'Upper-Uniform', 'Uniform'])

# Gera um arquivo CSV com os resultados.
if beta < 1:
  key = '0_' + str(beta).split('.')[1]

else:
  spt = str(beta).split('.')
  key = spt[0] + '_' + spt[1]

df.to_csv(f'beta{key}_n{size}.csv', index=False)
files.download(f'beta{key}_n{size}.csv')

Transform Dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Estimating Betas:   0%|          | 0/1000 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(df['Gamma'].mean(), df['Uniform'].mean())

0.15399069080511243 0.1538615909378363
